In [123]:
# 网页相关模块
import requests
import re

# 数据分析模块
import pandas as pd

# 数据可视化模块
import matplotlib as mpl
import matplotlib.pyplot as plt

# 可视化样式调节
mpl.style.use('seaborn')
plt.rcParams['font.sans-serif'] = ['.PingFang SC']  # 用来正常显示中文标签
mpl.rcParams['legend.fontsize'] = 16
mpl.rcParams['axes.titlesize'] = 18

In [124]:
# 爬取网页
res = requests.get('http://www.shuichan.cc/article_list-144-663.html')
res.encoding = 'gb2312'

soup = BeautifulSoup(res.text, 'html5lib')
all_tags = soup.findAll("a")
p = re.compile('武汉')

# 获取所有武汉白沙洲水产市场的网站
wuhan_sites = []
for n, tag in enumerate(all_tags):
    try:
        wuhan = p.search(tag['title'])
        if wuhan:
            wuhan_sites.append(tag['href'])
    except KeyError:
        pass


In [125]:
# 读取数据
price_tables = []
for each_site in wuhan_sites[0:3]:
    data = pd.read_html('http://www.shuichan.cc/%s' % each_site)
    price_table = data[8]
    
    # 将第一行作为列名称
    new_header = price_table.iloc[0]

    price_table.columns = new_header

    price_table = price_table[1:]
    
    price_table = price_table.set_index('分类规格')
    
    price_tables.append(price_table)

In [126]:
total_table = pd.concat(price_tables, axis=1, join='outer')
total_table

,12月14日,12月10日,12月2日,12月10日,12月2日,11月30日
分类规格,,,,,,
25-34克,40元/斤,39元/斤,36元/斤,39元/斤,36元/斤,35元/斤
35-44克,48元/斤,48元/斤,44元/斤,48元/斤,44元/斤,43元/斤
45-54克,55元/斤,55元/斤,48元/斤,55元/斤,48元/斤,48元/斤
55-64克,61元/斤,60元/斤,56元/斤,60元/斤,56元/斤,55元/斤


In [127]:
for i in total_table:
    print(i)

12月14日
12月10日
12月2日
12月10日
12月2日
11月30日


In [128]:
# 去掉单位
for i in total_table:
    p = total_table[i].str.split('元', expand = True)[0]
    price_table[i] = p

AttributeError: 'DataFrame' object has no attribute 'str'

In [ ]:
price_table = price_table.applymap(int)

In [ ]:
price_table

In [ ]:
price_table.T.plot(figsize=[16,10], fontsize=16, title='价格变动')